In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2 
import os
from tqdm import tqdm
import json
import time
import random
import pickle
import keras
import scipy

In [4]:
n=500

In [7]:
df=pd.read_csv("E:\capstone\code\WLASL-master\WLASL-master\start_kit\data.csv")
words=df.gloss.unique()
len(words)

2000

In [8]:
ed=np.zeros((n,n))
for i in range(ed.shape[0]):
    ed[i][i]=1
ed=pd.DataFrame(ed)
ed.head()
ed['gloss']=words[:n]
ed.head()

,0,1,2,3,4,5,6,7,8,9,...,491,492,493,494,495,496,497,498,499,gloss
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,book
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,drink
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,computer
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,before
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,chair


In [10]:
x_train=[]
x_test=[]
x_val=[]
y_train=[]
y_test=[]
y_val=[]
y_words_train=[]
y_words_test=[]
y_words_val=[]


for word in tqdm(words[:n]): 

    for split in os.listdir(r'E:\capstone\code\WLASL-master\WLASL-master\start_kit\preprocessed_vid_by_word_48_64 16.09.58\{}'.format(word)):
        if split==".DS_Store" or split=='._.DS_Store':
            continue
        try:
            for v in os.listdir(r'E:\capstone\code\WLASL-master\WLASL-master\start_kit\preprocessed_vid_by_word_48_64 16.09.58\{}\{}'.format(word,split)):
                y_ind=ed[ed['gloss']==word].values[:,:-1].reshape((n))
            
                x_ind=np.load(r'E:\capstone\code\WLASL-master\WLASL-master\start_kit\preprocessed_vid_by_word_48_64 16.09.58\{}\{}\{}'.format(word,split,v))
                gray_scale=[]
                y_temp=[]
                prev_frame=np.zeros(2)
                for curr_frame in x_ind:
                    curr_frame=cv2.cvtColor(curr_frame, cv2.COLOR_BGR2GRAY)
                    if prev_frame.shape[0]==2:
                        if(split=="train" or split=="val" ):
                    
                            x_train.append(curr_frame.reshape(128,96,1))
                            y_train.append(y_ind)
                            y_words_train.append(word)
                        
                        elif(split=="test"):
                            x_test.append(curr_frame.reshape(128,96,1))
                            y_test.append(y_ind)
                            y_words_test.append(word)
                        prev_frame=curr_frame
                        continue
                    A=prev_frame.reshape(128*96*1)
                    B=curr_frame.reshape(128*96*1)
                    sim=scipy.stats.pearsonr(A,B)[0]
                    if(sim<.95):
                        
                        prev_frame=curr_frame
                
                        if(split=="train" or split=="val"):
                    
                            x_train.append(curr_frame.reshape(128,96,1))
                            y_train.append(y_ind)
                            y_words_train.append(word)
                        
                        elif(split=="test"):
                            x_test.append(curr_frame.reshape(128,96,1))
                            y_test.append(y_ind)
                            y_words_test.append(word)
               
                
        except Exception as e:
            print("Error:" ,e ,"file",i)
        

100%|████████████████████████████████████████████████████████████████████████████████| 500/500 [04:24<00:00,  1.89it/s]


In [11]:
x_train=np.array(x_train)
x_test=np.array(x_test)
x_val=np.array(x_val)
y_train=np.array(y_train)
y_test=np.array(y_test)
y_val=np.array(y_val)

In [12]:
from sklearn.utils import shuffle
x_train, y_train,y_words_train = shuffle(x_train, y_train,y_words_train,random_state=10)

In [13]:
from sklearn.model_selection import train_test_split
x_train1, x_test1, y_train1, y_test1 = train_test_split(x_train, y_train, test_size=0.30,random_state=10)

In [15]:
model = keras.models.load_model(r'capstone_model_96_128_custom_cnn-with-removing_highly_corelated_data_final_500_v1.h5')

In [33]:
x_train=x_train/255
pred=model.predict(x_train)

In [37]:
res1=np.argmax(y_train,axis=1)
res=np.argmax(pred,axis=1)

In [56]:
res3=[]
res4=[]
for i in res1:
    res3.append(words[i])
for i in res:
    res4.append(words[i])

In [57]:
res3=np.asarray(res3)
res4=np.asarray(res4)

In [58]:
(res3==res4).sum()/res.shape[0]

0.8992814356938749

In [42]:
from sklearn.metrics import top_k_accuracy_score
top_k_accuracy_score(res1, y_score,k=2)

ImportError: cannot import name 'top_k_accuracy_score' from 'sklearn.metrics' (C:\Users\HP\anaconda3\lib\site-packages\sklearn\metrics\__init__.py)

In [59]:
from sklearn.metrics import classification_report
print(classification_report(res3,res4))

              precision    recall  f1-score   support

       about       0.89      0.94      0.91       188
      accept       0.88      0.89      0.88       120
    accident       0.94      0.89      0.91       229
       adult       0.88      0.86      0.87        96
      africa       0.90      0.93      0.92       188
       after       0.86      0.97      0.91       158
   afternoon       0.95      0.91      0.93       171
       again       0.98      0.91      0.94       188
         age       0.89      0.83      0.86       114
         ago       0.89      0.82      0.85        49
         all       0.82      0.95      0.88       337
       allow       0.80      0.92      0.85       119
       alone       0.92      0.94      0.93        78
      always       0.87      0.85      0.86       109
     america       0.97      0.96      0.96       252
       angel       0.94      0.94      0.94       143
      animal       0.95      0.95      0.95       174
      answer       0.94    

In [67]:
x=classification_report(res,res1)

In [70]:
x=pd.DataFrame(classification_report(res,res1, digits=2,
                                        output_dict=True))

In [71]:
pre=np.asarray(x.iloc[1,1:])

In [83]:
np.sum(np.sort(pre)[202:])/300

0.9409371904504229

In [64]:
def classification_report_csv(report):
    report_data = []
    lines = report.split('\n')
    for line in lines[2:-3]:
        row = {}
        row_data = line.split('      ')
        row['class'] = row_data[0]
        row['precision'] = float(row_data[1])
        row['recall'] = float(row_data[2])
        row['f1_score'] = float(row_data[3])
        row['support'] = float(row_data[4])
        report_data.append(row)
    dataframe = pd.DataFrame.from_dict(report_data)
    return dataframe

In [66]:
x

'              precision    recall  f1-score   support\n\n           0       0.83      0.94      0.88       125\n           1       0.92      0.89      0.90       343\n           2       0.94      0.92      0.93       458\n           3       0.91      0.74      0.82       258\n           4       0.91      0.88      0.90       196\n           5       0.90      0.88      0.89       290\n           6       0.89      0.78      0.83       153\n           7       0.90      0.93      0.92       197\n           8       0.95      0.66      0.78       215\n           9       0.91      0.95      0.93       123\n          10       0.86      0.92      0.89       132\n          11       0.79      0.95      0.87       159\n          12       0.93      0.92      0.93       267\n          13       0.87      0.88      0.88       147\n          14       0.95      0.86      0.90       266\n          15       0.93      0.97      0.95       315\n          16       0.96      0.93      0.95       295\n       